In [ ]:
import numpy as np

class Graph():
  """ Define a computation graph, which contains
        placeholders: to store the input data
        variables: to store the network parameters
        constants: some static data
        operations: the mathmatical operations for each neural network layer
  """
  def __init__(self):
    """
       Please define attributes for placeholders, variables, constants, operations,
       using lists.
    """
    self.operations = []
    self.placeholders = []
    self.variables = []
    self.constants = []

  def as_default(self):
    """
      define a global default computation graph
    """
    global _default_graph
    _default_graph = self
    
  def add_operation(self, op):
    """
      add op to operations
    """
    self.operations.append(op)

  def add_placeholder(self, holder):
    """
      add holder to placeholders
    """
    self.placeholders.append(holder)
  
  def add_variable(self, var):
    """
      add var to variables
    """
    self.variables.append(var)
    
  def add_constant(self, c):
    """
      add c to constants
    """
    self.constants.append(c)

class Operation():
  """
    Define network operation node. It should contain input nodes and one output node.
  """
  def __init__(self, input_nodes=None):
    """
      Define two attributes here: 
        input_nodes
        output
      Add the current Operation node to _default_graph's operations
    """
    self.input_nodes = input_nodes
    self.output = None
    
    # Append operation to the list of operations of the default graph
    _default_graph.add_operation(self)

  def forward(self):
    pass

  def backward(self):
    pass


class BinaryOperation(Operation):
  """
    define binary operations
  """
  def __init__(self, a, b):
    """
      a, b are input nodes. please initialize
    """
    super().__init__([a, b])

class add(BinaryOperation):
  """
  Computes a + b, element-wise
  """
  def forward(self, a, b):
    return a + b

  def backward(self, upstream_grad):
    raise NotImplementedError

class multiply(BinaryOperation):
  """
  Computes a * b, element-wise
  """
  def forward(self, a, b):
    return a * b

  def backward(self, upstream_grad):
    raise NotImplementedError

class divide(BinaryOperation):
  """
  Returns the true division of the inputs, element-wise
  """
  def forward(self, a, b):
    return np.true_divide(a, b)

  def backward(self, upstream_grad):
    raise NotImplementedError

class matmul(BinaryOperation):
  """
  Multiplies matrix a by matrix b, producing a * b
  """
  def forward(self, a, b):
    """
      using numpy.dot to perform matrix multiplication on a and b
    """    
    return a.dot(b)

  def backward(self, upstream_grad):
    raise NotImplementedError
  
class Placeholder():
  """
    define placeholder. It should contain a value attribute to store value
  """
  def __init__(self):
    """
      initialize the value to None, add the current node to default graph's placeholder
    """    
    self.value = None
    _default_graph.add_placeholder(self)

class Constant():
  """
    Define a constant node
  """
  def __init__(self, value=None):
    """
      define internal __value to store the value, add the current node to graph's constant
    """    
    self.__value = value
    _default_graph.add_constant(self)

  @property
  def value(self):
    """
      return the internal value
    """
    return self.__value

  @value.setter
  def value(self, value):
    raise ValueError("Cannot reassign value.")
  
class Variable():
  """
    define a variable node (for parameter) with initial_value
  """
  def __init__(self, initial_value=None):
    """
      assign initial_value to value, add the current node to graph's variables
    """    
    self.value = initial_value
    _default_graph.add_variable(self)

def topology_sort(operation):
  """
    implement topological sort to order the operations, starting from current node
  """
  ordering = []
  visited_nodes = set()

  def recursive_helper(node):
    """
      for each Operation node (using isinstance)
      recursively find the incoming nodes, visit them first and add node to visited nodes. 
    """    
    if isinstance(node, Operation):
      for input_node in node.input_nodes:
        if input_node not in visited_nodes:
          recursive_helper(input_node)

    visited_nodes.add(node)
    ordering.append(node)

  # start recursive depth-first search
  recursive_helper(operation)

  return ordering

# session = Session()
# output = session.run(some_operation, {
#     X: train_X # [1,2,...,n_features]
# })

class Session():
  """
    A session provides a context to run the computation graph
  """
  def run(self, operation, feed_dict={}):
    """
      apply topological sort on the computation graph starting from the operation node
      operation is the final operation node
      feed_dict: a dictionary that maps Placeholder to actual data value (in numpy)
      if a node is a placeholder, it should take value from feed_dict, 
      if a node is variable or constant, it just use the node's value
      it a node is an operation, it should get the node's input_nodes, and then apply forward
    """
    nodes_sorted = topology_sort(operation)

    for node in nodes_sorted:
      if type(node) == Placeholder:
        node.output = feed_dict[node]
      elif type(node) == Variable or type(node) == Constant:
        node.output = node.value
      else:
        inputs = [node.output for node in node.input_nodes]
        node.output = node.forward(*inputs)

    return operation.output

In [8]:
# create default graph
Graph().as_default()

# construct computational graph by creating some nodes
# implement a simple network for y = a * x + b
a = Constant(np.array([2.0, 1.5]))
b = Constant(0.5)
x = Placeholder()
x2 = matmul(a, x)
y = add(x2, b)

x_data = np.array([0.5, 0.8])
input_data = {x: x_data}

# create a session object
session = Session()

# run computational graph to compute the output for 'res'
out = session.run(y, input_data)
print(out)

2.7
